In [1]:
#データの読み込み
import pandas as pd
import os
import matplotlib.pyplot as plt

os.chdir("C:/Users/***/Downloads/all")
df = pd.read_csv("train.csv") 

C:\Users\haruo\Anaconda3\envs\tensorflow\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
C:\Users\haruo\Anaconda3\envs\tensorflow\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
#欠損値の確認
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
#特徴量の整形と追加
import numpy as np
df = pd.read_csv("train.csv").replace("male",0).replace("female",1)
df = df.replace("C",0).replace("Q",1).replace("S",2)

df["Family"] = df["SibSp"] + df["Parch"] + 1
df["IsAlone"] = 1
df["IsAlone"].loc[df["Family"] > 1 ] = 0


combine = [df]
for train in combine:
    train["Title"] = train.Name.str.extract(' ([A-Za-z]+).', expand=False) 
    
for dataset in combine:
    dataset["Title"] = dataset["Title"].replace(['Lady', 'Countess','Capt', 
                                                 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset["Title"] = dataset["Title"].replace('Mlle', 'Miss')
    dataset["Title"] = dataset["Title"].replace('Ms', 'Miss')
    dataset["Title"] = dataset["Title"].replace('Mme', 'Mrs')

title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    
for train in combine: 
    train['Cabin_Lett'] = train['Cabin'].apply(lambda x: str(x)[0]) 
    train['Cabin_Lett'] = train['Cabin_Lett'].apply(lambda x: str(x)) 
    train['Cabin_Lett'] = np.where((train['Cabin_Lett']).isin([ 'F', 'E', 'D', 'C', 'B', 'A'])
                                   ,train['Cabin_Lett'], np.where((train['Cabin_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']), '0','0'))
train['Cabin_Lett']=train['Cabin_Lett'].replace("A",1).replace("B",2).replace("C",1).replace("0",0).replace("D",2).replace("E",2).replace("F",1)

for train in combine: 
        train['Ticket_Lett'] = train['Ticket'].apply(lambda x: str(x)[0])
        train['Ticket_Lett'] = train['Ticket_Lett'].apply(lambda x: str(x)) 
        train['Ticket_Lett'] = np.where((train['Ticket_Lett']).isin(['1', '2', '3', 'S', 'P', 'C', 'A']), train['Ticket_Lett'], np.where((train['Ticket_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']), '0','0')) 
        train['Ticket_Len'] = train['Ticket'].apply(lambda x: len(x)) 
train['Ticket_Lett']=train['Ticket_Lett'].replace("1",1).replace("2",2).replace("3",3).replace("0",0).replace("S",3).replace("P",0).replace("C",3).replace("A",3)
    
#特徴量の削除
df_2 = df.drop(["Name","SibSp","Parch","Ticket","Cabin"],axis=1)
df_2["Age"] = df_2["Age"].fillna(df_2["Age"].mean())
df_2["Embarked"] = df_2["Embarked"].fillna("S")
df_2 = df_2.replace("C",0).replace("Q",1).replace("S",2)
df_3 = df_2

#sklearn用にデータ整形
X_train, y_train= df_3.values[:,2:], df_3.values[:,1]

C:\Users\haruo\Anaconda3\envs\tensorflow\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
#アンサンブル分類器
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.externals import six
from sklearn.base import clone
from sklearn.pipeline import _name_estimators
import operator


class MajorityVoteClassifier(BaseEstimator, 
                             ClassifierMixin):
    """ A majority vote ensemble classifier

    Parameters
    ----------
    classifiers : array-like, shape = [n_classifiers]
      Different classifiers for the ensemble

    vote : str, {'classlabel', 'probability'} (default='label')
      If 'classlabel' the prediction is based on the argmax of
        class labels. Else if 'probability', the argmax of
        the sum of probabilities is used to predict the class label
        (recommended for calibrated classifiers).

    weights : array-like, shape = [n_classifiers], optional (default=None)
      If a list of `int` or `float` values are provided, the classifiers
      are weighted by importance; Uses uniform weights if `weights=None`.

    """
    def __init__(self, classifiers, vote='classlabel', weights=None):

        self.classifiers = classifiers
        self.named_classifiers = {key: value for key, value
                                  in _name_estimators(classifiers)}
        self.vote = vote
        self.weights = weights

    def fit(self, X, y):
        """ Fit classifiers.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Matrix of training samples.

        y : array-like, shape = [n_samples]
            Vector of target class labels.

        Returns
        -------
        self : object

        """
        if self.vote not in ('probability', 'classlabel'):
            raise ValueError("vote must be 'probability' or 'classlabel'"
                             "; got (vote=%r)"
                             % self.vote)

        if self.weights and len(self.weights) != len(self.classifiers):
            raise ValueError('Number of classifiers and weights must be equal'
                             '; got %d weights, %d classifiers'
                             % (len(self.weights), len(self.classifiers)))

        # Use LabelEncoder to ensure class labels start with 0, which
        # is important for np.argmax call in self.predict
        self.lablenc_ = LabelEncoder()
        self.lablenc_.fit(y)
        self.classes_ = self.lablenc_.classes_
        self.classifiers_ = []
        for clf in self.classifiers:
            fitted_clf = clone(clf).fit(X, self.lablenc_.transform(y))
            self.classifiers_.append(fitted_clf)
        return self

    def predict(self, X):
        """ Predict class labels for X.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Matrix of training samples.

        Returns
        ----------
        maj_vote : array-like, shape = [n_samples]
            Predicted class labels.
            
        """
        if self.vote == 'probability':
            maj_vote = np.argmax(self.predict_proba(X), axis=1)
        else:  # 'classlabel' vote

            #  Collect results from clf.predict calls
            predictions = np.asarray([clf.predict(X)
                                      for clf in self.classifiers_]).T

            maj_vote = np.apply_along_axis(
                                      lambda x:
                                      np.argmax(np.bincount(x,
                                                weights=self.weights)),
                                      axis=1,
                                      arr=predictions)
        maj_vote = self.lablenc_.inverse_transform(maj_vote)
        return maj_vote

    def predict_proba(self, X):
        """ Predict class probabilities for X.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.

        Returns
        ----------
        avg_proba : array-like, shape = [n_samples, n_classes]
            Weighted average probability for each class per sample.

        """
        probas = np.asarray([clf.predict_proba(X)
                             for clf in self.classifiers_])
        avg_proba = np.average(probas, axis=0, weights=self.weights)
        return avg_proba

    def get_params(self, deep=True):
        """ Get classifier parameter names for GridSearch"""
        if not deep:
            return super(MajorityVoteClassifier, self).get_params(deep=False)
        else:
            out = self.named_classifiers.copy()
            for name, step in six.iteritems(self.named_classifiers):
                for key, value in six.iteritems(step.get_params(deep=True)):
                    out['%s__%s' % (name, key)] = value
            return out

In [6]:
#パイプラインの構築
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

clf1 = LogisticRegression(penalty='l2', 
                          C=10.0,
                          random_state=1)

clf2 = RandomForestClassifier(criterion="gini", 
                              n_estimators=100, 
                              random_state=1)

clf3 = KNeighborsClassifier(n_neighbors=1,
                            p=2,
                            metric='minkowski')

clf4 = SVC(kernel='rbf',
           gamma=0.01,
           C=10.0, random_state=1,probability=True)

pipe1 = Pipeline([['sc', StandardScaler()],
                  ['clf', clf1]])
pipe3 = Pipeline([['sc', StandardScaler()],
                 ['clf', clf3]])
pipe4 = Pipeline([['sc', StandardScaler()],
                  ['clf', clf4]])

clf_labels = ['Logistic regression', 'Random Forest','KNN', 'SVM']

print('10-fold cross validation:\n')
for clf, label in zip([pipe1, clf2,pipe3, pipe4], clf_labels):
    scores = cross_val_score(estimator=clf,
                             X=X_train,
                             y=y_train,
                             cv=10,
                             scoring='roc_auc')
    print("ROC AUC: %0.2f (+/- %0.2f) [%s]"
          % (scores.mean(), scores.std(), label))

10-fold cross validation:

ROC AUC: 0.87 (+/- 0.03) [Logistic regression]
ROC AUC: 0.88 (+/- 0.06) [Random Forest]
ROC AUC: 0.75 (+/- 0.06) [KNN]
ROC AUC: 0.85 (+/- 0.05) [SVM]


In [7]:
#多数決分類器
mv_clf = MajorityVoteClassifier(classifiers=[pipe1, clf2,pipe3, pipe4])

clf_labels = ['Logistic regression', 'XGboost', 'KNN', 'SVM','Majority voting']
all_clf = [pipe1, clf2,pipe3, pipe4, mv_clf]

for clf, label in zip(all_clf, clf_labels):
    scores = cross_val_score(estimator=clf,
                             X=X_train,
                             y=y_train,
                             cv=10,
                             scoring='roc_auc')
    print("ROC AUC: %0.2f (+/- %0.2f) [%s]"
          % (scores.mean(), scores.std(), label))

ROC AUC: 0.87 (+/- 0.03) [Logistic regression]
ROC AUC: 0.88 (+/- 0.06) [XGboost]
ROC AUC: 0.75 (+/- 0.06) [KNN]
ROC AUC: 0.85 (+/- 0.05) [SVM]
ROC AUC: 0.88 (+/- 0.05) [Majority voting]


In [ ]:
#グリッドサーチによるパラメータの調整
from sklearn.model_selection import GridSearchCV

params = {'decisiontreeclassifier__max_depth': [1, 2],
          'pipeline-1__clf__C': [0.001, 0.01, 1.0, 10.0,100.0],
          'pipeline-3__clf__C': [0.001, 0.01, 1.0, 10.0 ,100.0],
          'pipeline-3__clf__gamma': [0.001,0.01,1.0,10.0,100.0]}

grid = GridSearchCV(estimator=mv_clf,
                    param_grid=params,
                    cv=10,
                    scoring='roc_auc')
grid.fit(X_train, y_train)

for r, _ in enumerate(grid.cv_results_['mean_test_score']):
    print("%0.3f +/- %0.2f %r"
          % (grid.cv_results_['mean_test_score'][r], 
             grid.cv_results_['std_test_score'][r] / 2.0, 
             grid.cv_results_['params'][r]))


print('Best parameters: %s' % grid.best_params_)
print('Accuracy: %.2f' % grid.best_score_)    

In [9]:
#テストデータを作る
os.chdir("C:/Users/***/Downloads/all")
df = pd.read_csv("test.csv")

#データの整形
df_test = pd.read_csv("test.csv").replace("male",0).replace("female",1)
df_test = df_test.replace("C",0).replace("Q",1).replace("S",2)
df_test["Family"] = df_test["SibSp"] + df_test["Parch"] + 1
df_test["IsAlone"] = 1
df_test["IsAlone"].loc[df_test["Family"] > 1] = 0

combine = [df_test]
for train in combine:
    train["Title"] = train.Name.str.extract(' ([A-Za-z]+).', expand=False) 
    
for dataset in combine:
    dataset["Title"] = dataset["Title"].replace(['Lady', 'Countess','Capt', 
                                                 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset["Title"] = dataset["Title"].replace('Mlle', 'Miss')
    dataset["Title"] = dataset["Title"].replace('Ms', 'Miss')
    dataset["Title"] = dataset["Title"].replace('Mme', 'Mrs')

title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

for train in combine: 
    train['Cabin_Lett'] = train['Cabin'].apply(lambda x: str(x)[0]) 
    train['Cabin_Lett'] = train['Cabin_Lett'].apply(lambda x: str(x)) 
    train['Cabin_Lett'] = np.where((train['Cabin_Lett']).isin([ 'F', 'E', 'D', 'C', 'B', 'A'])
                                   ,train['Cabin_Lett'], np.where((train['Cabin_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']), '0','0'))
train['Cabin_Lett']=train['Cabin_Lett'].replace("A",1).replace("B",2).replace("C",1).replace("0",0).replace("D",2).replace("E",2).replace("F",1)

for train in combine: 
        train['Ticket_Lett'] = train['Ticket'].apply(lambda x: str(x)[0])
        train['Ticket_Lett'] = train['Ticket_Lett'].apply(lambda x: str(x)) 
        train['Ticket_Lett'] = np.where((train['Ticket_Lett']).isin(['1', '2', '3', 'S', 'P', 'C', 'A']), train['Ticket_Lett'], np.where((train['Ticket_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']), '0','0')) 
        train['Ticket_Len'] = train['Ticket'].apply(lambda x: len(x)) 
train['Ticket_Lett']=train['Ticket_Lett'].replace("1",1).replace("2",2).replace("3",3).replace("0",0).replace("S",3).replace("P",0).replace("C",3).replace("A",3)
    

df_test2 = df_test.drop(["Name","SibSp","Parch","Ticket","Cabin"],axis=1)
df_test2["Age"] = df_test2["Age"].fillna(df_test2["Age"].mean())
df_test2["Embarked"] = df_test2["Embarked"].fillna("S")
df_test2 = df_test2.replace("C",0).replace("Q",1).replace("S",2)
df_test2["Fare"] = df_test2["Fare"].fillna(df_test2["Fare"].mean())
df_test3 = df_test2

X_test= df_test3.values[:,1:]

df_test3

C:\Users\haruo\Anaconda3\envs\tensorflow\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,PassengerId,Pclass,Sex,Age,Fare,Embarked,Family,IsAlone,Title,Cabin_Lett,Ticket_Lett,Ticket_Len
0,892,3,0,34.50000,7.8292,1,1,1,1.0,0,3,6
1,893,3,1,47.00000,7.0000,2,2,0,3.0,0,3,6
2,894,2,0,62.00000,9.6875,1,1,1,1.0,0,2,6
3,895,3,0,27.00000,8.6625,2,1,1,1.0,0,3,6
4,896,3,1,22.00000,12.2875,2,3,0,3.0,0,3,7
5,897,3,0,14.00000,9.2250,2,1,1,1.0,0,0,4
6,898,3,1,30.00000,7.6292,1,1,1,2.0,0,3,6
7,899,2,0,26.00000,29.0000,2,3,0,1.0,0,2,6
8,900,3,1,18.00000,7.2292,0,1,1,3.0,0,2,4
9,901,3,0,21.00000,24.1500,2,3,0,1.0,0,3,9


In [10]:
#アンサンブルの結果予測
for clf, label in zip(all_clf,clf_labels):
    y_predA = clf.fit(X_train, y_train).predict(X_test)
y_predA = y_predA.astype(np.int64)

# PassengerId
PassengerId = np.array(df_test3["PassengerId"]).astype(np.int)
result = pd.DataFrame(y_predA, PassengerId, columns = ["Survived"])
# csvとして書き出し
result.to_csv("result_ensemble1.csv", index_label = ["PassengerId"])

C:\Users\haruo\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
